[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/azad-academy/personalized-diffusion-tutorial/blob/main/personalized_generation_tutorial.ipynb)

# Personalized Diffusion For Customized Image Synthesis 

#### Author : J. Rafid Siddiqui (jrs@azaditech.com)

![Stable Diffusion Model Image](https://github.com/azad-academy/personalized-diffusion-tutorial/blob/main/cover.png)

                                Personalization/Customization of Stable Diffusion Models.
 

### Getting the Repo

In [26]:
!git clone https://github.com/azad-academy/personalized-diffusion-tutorial personalized-diffusion
%cd personalized-diffusion

Cloning into 'personalized-diffusion'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 76 (delta 8), reused 70 (delta 5), pack-reused 0
Unpacking objects: 100% (76/76), done.
/content/personalized-diffusion


# Pre-requisites, Imports and Initializers

In [6]:
#@markdown # Pre-requisites
from IPython.display import clear_output
from subprocess import getoutput
from IPython.utils import capture

!git clone https://github.com/TheLastBen/diffusers
!pip install git+https://github.com/TheLastBen/diffusers
%pip install transformers
%pip install ftfy
%pip install accelerate==0.12.0
%pip install bitsandbytes
%pip install -q triton==2.0.0.dev20220701
clear_output()
print("Done. Installed all packages")

Done. Installed all packages


In [7]:
#@markdown # xformers (Needed for choosing the optimal setting for the respective GPU on Google Colab)

from subprocess import getoutput
from IPython.display import HTML
from IPython.display import clear_output
import time

s = getoutput('nvidia-smi')
if 'T4' in s:
  gpu = 'T4'
elif 'P100' in s:
  gpu = 'P100'
elif 'V100' in s:
  gpu = 'V100'
elif 'A100' in s:
  gpu = 'A100'

while True:
    try: 
        gpu=='T4'or gpu=='P100'or gpu=='V100'or gpu=='A100'
        break
    except:
        pass
    print('[1;31mit seems that your GPU is not supported at the moment')
    time.sleep(5)

if (gpu=='T4'):
  %pip install https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/T4/xformers-0.0.13.dev0-py3-none-any.whl
  
elif (gpu=='P100'):
  %pip install https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/P100/xformers-0.0.13.dev0-py3-none-any.whl

elif (gpu=='V100'):
  %pip install https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/V100/xformers-0.0.13.dev0-py3-none-any.whl

elif (gpu=='A100'):
  %pip install https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/A100/xformers-0.0.13.dev0-py3-none-any.whl  

clear_output()
print('[1;32mDONE !')

DONE !


In [27]:
#Installing dependent models
!python setup_pre-models.py

Cloning into 'stable-diffusion'...
remote: Enumerating objects: 313, done.
remote: Total 313 (delta 0), reused 0 (delta 0), pack-reused 313
Receiving objects: 100% (313/313), 42.63 MiB | 46.79 MiB/s, done.
Resolving deltas: 100% (103/103), done.
Cloning into 'stable-diffusion-webui'...
remote: Enumerating objects: 6030, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 6030 (delta 80), reused 72 (delta 41), pack-reused 5894
Receiving objects: 100% (6030/6030), 21.54 MiB | 45.77 MiB/s, done.
Resolving deltas: 100% (4220/4220), done.
/content
Cloning into 'taming-transformers'...
remote: Enumerating objects: 1335, done.
remote: Total 1335 (delta 0), reused 0 (delta 0), pack-reused 1335
Receiving objects: 100% (1335/1335), 409.77 MiB | 50.54 MiB/s, done.
Resolving deltas: 100% (278/278), done.
Cloning into 'CLIP'...
remote: Enumerating objects: 236, done.
remote: Total 236 (delta 0), reused 0 (delta 0), pack-reused 236
Rec

## HuggingFace Login

In [13]:
#HuggingFace Login

from huggingface_hub import notebook_login
notebook_login()


Login successful
Your token has been saved to /root/.huggingface/token


### Mounting the Google Drive
Execute this, If you would want to keep the session active and not login or download model again and again.

In [14]:
from google.colab import drive
drive.mount('/content/drive/')

cache_dir = "/content/drive/MyDrive/Colab Notebooks/weights" #@param {type:"string"}
cache_drive = True

Mounted at /content/drive/


In [15]:
from huggingface_hub import hf_hub_download
print("Downloading the model, Please Wait...")
model_path = hf_hub_download(repo_id="CompVis/stable-diffusion-v-1-4-original", filename="sd-v1-4.ckpt",cache_dir="/content/drive/MyDrive/Colab Notebooks/weights")
print("Downloaded Model: "+ model_path)


Downloading:   0%|          | 0.00/4.27G [00:00<?, ?B/s]

Downloaded Model: /content/drive/MyDrive/Colab Notebooks/weights/models--CompVis--stable-diffusion-v-1-4-original/snapshots/0834a76f88354683d3f7ef271cadd28f4757a8cc/sd-v1-4.ckpt


## Texual Inversion

### Generating Customized Images using the Learned Embeddings:

In [16]:
import os
from IPython.display import clear_output
from subprocess import getoutput
from IPython.utils import capture
import time

EMBEDDING__NAME="shifu" #@param{type: 'string'}

MODEL_PATH='' #@param{type:'string'}         

if model_path:
  MODEL_PATH=model_path

In [ ]:
#Running the server
%cd /content/personalized-diffusion/stable-diffusion/
!python ../run.py ""

/content/personalized-diffusion/stable-diffusion
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Loading weights [7460a6fa] from /content/personalized-diffusion/models/model.ckpt
Global Step: 470000


In [63]:
#model_path = "/content/drive/MyDrive/Colab Notebooks/weights/models--CompVis--stable-diffusion-v-1-4-original/snapshots/0834a76f88354683d3f7ef271cadd28f4757a8cc/sd-v1-4.ckpt"
!cp "{model_path}" /content/personalized-diffusion/models/model.ckpt

In [41]:
#!cp "/usr/local/lib/python3.7/dist-packages/gradio/strings.py" ./strings.py
#!cp ./strings.py "/usr/local/lib/python3.7/dist-packages/gradio/strings.py"

In [25]:
%cd /content/
!rm -rf /content/personalized-diffusion

/content


## Customized Image Generation with DreamBooth


In [66]:
with capture.capture_output() as cap: 
  os.chdir("/content/personalized-diffusion/stable-diffusion-webui/")
  time.sleep(1)
  os.system("wget -O webui.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/webui.py")
  os.system("sed -i 's@gpu_call).*@gpu_call) \n        demo.queue(concurrency_count=111500)@' /content/personalized-diffusion/stable-diffusion-webui/webui.py")

## Downloading the fine-tuned models


### 

### 

### Loading an Image to Image Model

# References


[1] Robin Rombach, Andreas Blattmann, Dominik Lorenz, Patrick Esser, Björn Ommer, [Latent Diffusion Models](https://arxiv.org/abs/2112.10752), 2022

[2] Rinon Gal, Yuval Alaluf, Yuval Atzmon, Or Patashnik, Amit H. Bermano, Gal Chechik, Daniel Cohen-Or, [An Image is Worth One Word: Personalizing Text-to-Image Generation using Textual Inversion](https://arxiv.org/abs/2208.01618), 2022

[3] Nataniel Ruiz, Yuanzhen Li, Varun Jampani, Yael Pritch, Michael Rubinstein, Kfir Aberman, [DreamBooth: Fine Tuning Text-to-Image Diffusion Models for Subject-Driven Generation](https://arxiv.org/abs/2208.12242), 2022